### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [1]:
# imports
import pyspark
import pandas as pd

# create spark environment
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
# create pandas dataframe
p_df = pd.DataFrame({'language':['python', 'sql', 'html', 'ruby', 'c', 'scala']})
p_df

,language
0,python
1,sql
2,html
3,ruby
4,c
5,scala


In [3]:
# create spark dataframe
df = spark.createDataFrame(p_df)

In [4]:
# print schema
df.printSchema()

root
 |-- language: string (nullable = true)



In [5]:
# output shape
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

DataFrame shape:  6  x  1


In [6]:
# view first 5 records
df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|    html|
|    ruby|
|       c|
+--------+
only showing top 5 rows



### 2. Load the `mpg` dataset as a spark dataframe.

#### a. Create 1 column of output that contains a message like the one below for each record:

    The 1999 audi a4 has a 4 cylinder engine.

> Hint: You will need to concatenate values that already exist in the data with string literals

In [7]:
# import
from pydataset import data

# create dataframe
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [8]:
# imports
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat

# create string column
mpg.select(concat(lit("The "),
                  mpg.year,
                  lit(" "),
                  mpg.manufacturer,
                  lit(" "),
                  mpg.model,
                  lit(" has a "),
                  mpg.cyl,
                  lit(" cylinder engine."))
          .alias("message")
          ).show(5, truncate=False)

+-----------------------------------------+
|message                                  |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



#### b. Transform the trans column so that it only contains either manual or auto.

> Hint: Consider spark string methods and `when().otherwise()` chaining

In [9]:
# preview dataframe
mpg.show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|        a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|        a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|compact|
|        audi|a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|
+-----------

In [10]:
# import
from pyspark.sql.functions import when, regexp_extract, regexp_replace, col, round

mpg.select(mpg.trans, 
           when(mpg.trans.like("a%"), "auto")
           .otherwise("manual")
           .alias("trans_type")
          ).show(10)

+----------+----------+
|     trans|trans_type|
+----------+----------+
|  auto(l5)|      auto|
|manual(m5)|    manual|
|manual(m6)|    manual|
|  auto(av)|      auto|
|  auto(l5)|      auto|
|manual(m5)|    manual|
|  auto(av)|      auto|
|manual(m5)|    manual|
|  auto(l5)|      auto|
|manual(m6)|    manual|
+----------+----------+
only showing top 10 rows



### 3. Load the `tips` dataset as a spark dataframe.

#### a. What percentage of observations are smokers?
> Hint: `.groupBy()` and `.withColumn()` are useful functions here

In [11]:
# get tips data
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [12]:
# show proportion of smokers vs non-smokers
tips.groupBy("smoker").count().withColumn("percentage", 
                                         round(col("count")/tips.count(), 2)).show()

+------+-----+----------+
|smoker|count|percentage|
+------+-----+----------+
|    No|  151|      0.62|
|   Yes|   93|      0.38|
+------+-----+----------+



#### b. Create a column that contains the tip percentage
> Hint: `.withColumn()` is useful here

In [13]:
# add tip percentage column
tips.withColumn("tip_percentage", round(tips.tip/tips.total_bill, 2)).show(5)

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          0.06|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.16|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.17|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.14|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          0.15|
+----------+----+------+------+---+------+----+--------------+
only showing top 5 rows



#### c. Calculate the average tip percentage for each combination of sex and smoker.
> Hint: Chain additional functions off the answer to part b 

In [14]:
# import
from pyspark.sql.functions import avg
# get tip percentages
tips.withColumn("tip_percentage", tips.tip/tips.total_bill).groupBy("smoker", "sex").agg(avg("tip_percentage")).show()

+------+------+-------------------+
|smoker|   sex|avg(tip_percentage)|
+------+------+-------------------+
|    No|Female| 0.1569209707691836|
|    No|  Male| 0.1606687151291298|
|   Yes|  Male|0.15277117520248512|
|   Yes|Female|0.18215035269941032|
+------+------+-------------------+



In [15]:
# do the same using a pivot table
tips.withColumn("tip_percentage", tips.tip/tips.total_bill).groupBy("smoker").pivot("sex").agg(avg("tip_percentage")).show()

+------+-------------------+-------------------+
|smoker|             Female|               Male|
+------+-------------------+-------------------+
|    No| 0.1569209707691836| 0.1606687151291298|
|   Yes|0.18215035269941032|0.15277117520248512|
+------+-------------------+-------------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.

In [16]:
# get seattle weather data
from vega_datasets import data

weather = data.seattle_weather()
weather = spark.createDataFrame(weather)
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [22]:
# degF = (degC * 1.8) + 32
from pyspark.sql.functions import month, year, quarter, expr, sum as add, mean
# convert temp to fahrenheit
weather.withColumn("temp_max_F", expr("ROUND((temp_max*1.8)+32, 1)"))\
       .withColumn("temp_min_F", expr("ROUND((temp_min*1.8)+32, 1)"))\
       .show(5)

+-------------------+-------------+--------+--------+----+-------+----------+----------+
|               date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|
+-------------------+-------------+--------+--------+----+-------+----------+----------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|
+-------------------+-------------+--------+--------+----+-------+----------+----------+
only showing top 5 rows



- Which month has the most rain, on average?

In [23]:
weather.withColumn("month", month("date"))\
       .withColumn("year", year("date"))\
       .groupBy("month", "year").agg(add("precipitation")\
                                     .alias("total_monthly_precipitation")).show()

+-----+----+---------------------------+
|month|year|total_monthly_precipitation|
+-----+----+---------------------------+
|    6|2012|                       75.1|
|    2|2012|                       92.3|
|    5|2012|         52.199999999999996|
|    1|2012|         173.29999999999998|
|    3|2012|                      183.0|
|    4|2012|          68.09999999999998|
|    8|2012|                        0.0|
|   12|2012|                      174.0|
|   10|2012|         170.29999999999998|
|    9|2012|         0.8999999999999999|
|   11|2012|                      210.5|
|    7|2012|                       26.3|
|    6|2013|                       33.1|
|    1|2013|         105.69999999999997|
|    5|2013|          60.49999999999999|
|    2|2013|         40.300000000000004|
|    3|2013|                       69.7|
|    4|2013|         149.60000000000002|
|   12|2013|          42.39999999999999|
|    8|2013|                       34.4|
+-----+----+---------------------------+
only showing top

In [25]:
rainiest_month = (
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(add("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
    .first()
)
rainiest_month

Row(month=11, avg_monthly_rain=160.625)

November was the rainiest month, with an average precipitation of 160.625.

- Which year was the windiest?

In [30]:
windiest_month = (
    weather.withColumn("year", year("date")) # add year column
    .groupBy("year") # make each year a row
    .agg(add("wind").alias("total_yearly_wind")) # aggregate by total wind per year
    .sort(col("total_yearly_wind").desc()) # sort wind totals
    .first() # get windiest year
)
windiest_month

Row(year=2012, total_yearly_wind=1244.7)

2012 was the windiest year, with a wind total of 1244.7

- What is the most frequent type of weather in January?

In [38]:
weather.withColumn("month", month("date"))\ # create month column
.filter(col("month") == 1)\ # show only january records
.groupby("weather")\ # make each weather type a row 
.count()\ # aggregate by count
.sort(col("count").desc())\ # sort descending by count
.show(1) # show first record

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



- What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [ ]:
weather.withColumn("month", month("date"))\
.withColumn("year", year("date"))\
.groupby("month", "year")\
.agg(mean())

In [40]:
weather.filter(month("date") == 7)\
.filter(year("date") > 2012)\
.filter(year("date") < 2015)\
.filter(col("weather") == lit("sun"))\
.agg(\
    avg("temp_max").alias("average_high_temp"),
    avg("temp_min").alias("average_low_temp"),
)\
.show()

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|26.828846153846158|14.18269230769231|
+------------------+-----------------+



- What percentage of days were rainy in q3 of 2015?

In [44]:
weather.filter(year("date") == 2015)\
.filter(quarter("date") == 3)\
.select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))\
.agg(mean("rain")).show()

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



- For each year, find what percentage of days it rained (had non-zero precipitation).

In [45]:
weather.withColumn("year", year("date"))\ # add year column
.select(when(col("precipitation") > 0, 1).otherwise(0).alias("rain"), "year")\ # convert to binary based on whether or not it rained
.groupby("year")\ # one value per year
.agg(mean("rain")).show() # show pct of rainy days per year

+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

